# Capstone Project Part 1: Data Cleaning

In [1]:
# imports
import pandas as pd
import numpy as np
import re
import pickle
import os
import numpy as np
from category_encoders import TargetEncoder 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
import reverse_geocoder as rg
import pprint

In [2]:
#reading in csv 
tdor = pd.read_csv('../Data/processed_data/tdor.csv')

In [3]:
# # to make DataFrames easier to navigate through during cleaning/EDA
# pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_colwidth', None)

In [5]:
# rename columns without special symbols
tdor = tdor.rename(columns = {'Town/City/Municipality':'Location', 'State/Province':'State Province'})

In [6]:
# converting column names to lowercase and removing spaces
def convert_to_snake_case(df):
    #add a space between any lowercase-capital letter pair, then replace spaces with _, the all to lowercase
    new_cols = {col: re.sub(r"([a-z]{1})([A-Z]{1})", r"\1 \2", col).replace(" ", "_").lower() for col in df.columns}
    return df.rename(columns = new_cols, inplace = True)

convert_to_snake_case(tdor)

In [7]:
#taking out alphabetical and punctuation characters to make column numerical
tdor['age'] = tdor['age'].str.replace('about ','').str.replace('Under ','').str.replace('"','').str.replace('"','').str.replace('Approx. ','').str.replace('Under ','').str.split('-', expand=True).astype(float).mean(axis=1)

/var/folders/qn/34_2dqh118513z6jb88w8spw0000gn/T/ipykernel_39320/2958069660.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tdor['age'] = tdor['age'].str.replace('about ','').str.replace('Under ','').str.replace('"','').str.replace('"','').str.replace('Approx. ','').str.replace('Under ','').str.split('-', expand=True).astype(float).mean(axis=1)


In [8]:
# converting the date column to datetime
tdor['date'] = pd.to_datetime(tdor['date'].str.zfill(6), format = '%d-%b-%Y')
tdor['date'].head(2)

0   2022-01-01
1   2022-01-02
Name: date, dtype: datetime64[ns]

In [9]:
# creating a year column from the date column
tdor['year'] = pd.DatetimeIndex(tdor['date']).year

In [10]:
# tdor['longitude'].astype('float')
tdor['longitude'] = (tdor['longitude'].str.replace(',', ''))
tdor['latitude'] = (tdor['latitude'].str.replace(',', ''))

In [11]:
# converting longitude column to float
tdor['longitude']=tdor['longitude'].astype('float')

In [12]:
# converting latitude column to float
tdor['latitude']=tdor['latitude'].astype('float')

In [13]:
# dropping null values in longitude/latitude columns
tdor.dropna(subset=['longitude','longitude'],inplace=True)

In [14]:
def reverseGeocode(coordinates):
    result = rg.search(coordinates) 
    # result is a list containing ordered dictionary.
    return result


In [15]:
# dictionary to convert country codes to country names from reverseGeocode function
country_codes_to_country = {
	'AD': 'Andorra',
	'AE': 'United Arab Emirates',
	'AF': 'Afghanistan',
	'AG': 'Antigua & Barbuda',
	'AI': 'Anguilla',
	'AL': 'Albania',
	'AM': 'Armenia',
	'AN': 'Netherlands Antilles',
	'AO': 'Angola',
	'AQ': 'Antarctica',
	'AR': 'Argentina',
	'AS': 'American Samoa',
	'AT': 'Austria',
	'AU': 'Australia',
	'AW': 'Aruba',
	'AZ': 'Azerbaijan',
	'BA': 'Bosnia and Herzegovina',
	'BB': 'Barbados',
	'BD': 'Bangladesh',
	'BE': 'Belgium',
	'BF': 'Burkina Faso',
	'BG': 'Bulgaria',
	'BH': 'Bahrain',
	'BI': 'Burundi',
	'BJ': 'Benin',
	'BM': 'Bermuda',
	'BN': 'Brunei Darussalam',
	'BO': 'Bolivia',
	'BR': 'Brazil',
	'BS': 'Bahama',
	'BT': 'Bhutan',
	'BU': 'Burma (no longer exists)',
	'BV': 'Bouvet Island',
	'BW': 'Botswana',
	'BY': 'Belarus',
	'BZ': 'Belize',
	'CA': 'Canada',
	'CC': 'Cocos (Keeling) Islands',
	'CF': 'Central African Republic',
	'CG': 'Congo',
	'CH': 'Switzerland',
	'CI': 'Côte D\'ivoire (Ivory Coast)',
	'CK': 'Cook Iislands',
	'CL': 'Chile',
	'CM': 'Cameroon',
	'CN': 'China',
	'CO': 'Colombia',
	'CR': 'Costa Rica',
	'CS': 'Czechoslovakia (no longer exists)',
	'CU': 'Cuba',
	'CV': 'Cape Verde',
	'CX': 'Christmas Island',
	'CY': 'Cyprus',
	'CZ': 'Czech Republic',
	'DD': 'German Democratic Republic (no longer exists)',
	'DE': 'Germany',
	'DJ': 'Djibouti',
	'DK': 'Denmark',
	'DM': 'Dominica',
	'DO': 'Dominican Republic',
	'DZ': 'Algeria',
	'EC': 'Ecuador',
	'EE': 'Estonia',
	'EG': 'Egypt',
	'EH': 'Western Sahara',
	'ER': 'Eritrea',
	'ES': 'Spain',
	'ET': 'Ethiopia',
	'FI': 'Finland',
	'FJ': 'Fiji',
	'FK': 'Falkland Islands (Malvinas)',
	'FM': 'Micronesia',
	'FO': 'Faroe Islands',
	'FR': 'France',
	'FX': 'France, Metropolitan',
	'GA': 'Gabon',
	'GB': 'United Kingdom (Great Britain)',
	'GD': 'Grenada',
	'GE': 'Georgia',
	'GF': 'French Guiana',
	'GH': 'Ghana',
	'GI': 'Gibraltar',
	'GL': 'Greenland',
	'GM': 'Gambia',
	'GN': 'Guinea',
	'GP': 'Guadeloupe',
	'GQ': 'Equatorial Guinea',
	'GR': 'Greece',
	'GS': 'South Georgia and the South Sandwich Islands',
	'GT': 'Guatemala',
	'GU': 'Guam',
	'GW': 'Guinea-Bissau',
	'GY': 'Guyana',
	'HK': 'Hong Kong',
	'HM': 'Heard & McDonald Islands',
	'HN': 'Honduras',
	'HR': 'Croatia',
	'HT': 'Haiti',
	'HU': 'Hungary',
	'ID': 'Indonesia',
	'IE': 'Ireland',
	'IL': 'Israel',
	'IN': 'India',
	'IO': 'British Indian Ocean Territory',
	'IQ': 'Iraq',
	'IR': 'Islamic Republic of Iran',
	'IS': 'Iceland',
	'IT': 'Italy',
	'JM': 'Jamaica',
	'JO': 'Jordan',
	'JP': 'Japan',
	'KE': 'Kenya',
	'KG': 'Kyrgyzstan',
	'KH': 'Cambodia',
	'KI': 'Kiribati',
	'KM': 'Comoros',
	'KN': 'St. Kitts and Nevis',
	'KP': 'Korea, Democratic People\'s Republic of',
	'KR': 'Korea, Republic of',
	'KW': 'Kuwait',
	'KY': 'Cayman Islands',
	'KZ': 'Kazakhstan',
	'LA': 'Lao People\'s Democratic Republic',
	'LB': 'Lebanon',
	'LC': 'Saint Lucia',
	'LI': 'Liechtenstein',
	'LK': 'Sri Lanka',
	'LR': 'Liberia',
	'LS': 'Lesotho',
	'LT': 'Lithuania',
	'LU': 'Luxembourg',
	'LV': 'Latvia',
	'LY': 'Libyan Arab Jamahiriya',
	'MA': 'Morocco',
	'MC': 'Monaco',
	'MD': 'Moldova, Republic of',
	'MG': 'Madagascar',
	'MH': 'Marshall Islands',
	'ML': 'Mali',
	'MN': 'Mongolia',
	'MM': 'Myanmar',
	'MO': 'Macau',
	'MP': 'Northern Mariana Islands',
	'MQ': 'Martinique',
	'MR': 'Mauritania',
	'MS': 'Monserrat',
	'MT': 'Malta',
	'MU': 'Mauritius',
	'MV': 'Maldives',
	'MW': 'Malawi',
	'MX': 'Mexico',
	'MY': 'Malaysia',
	'MZ': 'Mozambique',
	'NA': 'Namibia',
	'NC': 'New Caledonia',
	'NE': 'Niger',
	'NF': 'Norfolk Island',
	'NG': 'Nigeria',
	'NI': 'Nicaragua',
	'NL': 'Netherlands',
	'NO': 'Norway',
	'NP': 'Nepal',
	'NR': 'Nauru',
	'NT': 'Neutral Zone (no longer exists)',
	'NU': 'Niue',
	'NZ': 'New Zealand',
	'OM': 'Oman',
	'PA': 'Panama',
	'PE': 'Peru',
	'PF': 'French Polynesia',
	'PG': 'Papua New Guinea',
	'PH': 'Philippines',
	'PK': 'Pakistan',
	'PL': 'Poland',
	'PM': 'St. Pierre & Miquelon',
	'PN': 'Pitcairn',
	'PR': 'Puerto Rico',
	'PT': 'Portugal',
	'PW': 'Palau',
	'PY': 'Paraguay',
	'QA': 'Qatar',
	'RE': 'Réunion',
	'RO': 'Romania',
	'RU': 'Russian Federation',
    'RS': 'Serbia',
	'RW': 'Rwanda',
	'SA': 'Saudi Arabia',
	'SB': 'Solomon Islands',
	'SC': 'Seychelles',
	'SD': 'Sudan',
	'SE': 'Sweden',
	'SG': 'Singapore',
	'SH': 'St. Helena',
	'SI': 'Slovenia',
	'SJ': 'Svalbard & Jan Mayen Islands',
	'SK': 'Slovakia',
	'SL': 'Sierra Leone',
	'SM': 'San Marino',
	'SN': 'Senegal',
	'SO': 'Somalia',
	'SR': 'Suriname',
	'ST': 'Sao Tome & Principe',
	'SU': 'Union of Soviet Socialist Republics (no longer exists)',
	'SV': 'El Salvador',
	'SY': 'Syrian Arab Republic',
	'SZ': 'Swaziland',
	'TC': 'Turks & Caicos Islands',
	'TD': 'Chad',
	'TF': 'French Southern Territories',
	'TG': 'Togo',
	'TH': 'Thailand',
	'TJ': 'Tajikistan',
	'TK': 'Tokelau',
	'TM': 'Turkmenistan',
	'TN': 'Tunisia',
	'TO': 'Tonga',
	'TP': 'East Timor',
	'TR': 'Turkey',
	'TT': 'Trinidad & Tobago',
	'TV': 'Tuvalu',
	'TW': 'Taiwan, Province of China',
	'TZ': 'Tanzania, United Republic of',
	'UA': 'Ukraine',
	'UG': 'Uganda',
	'UM': 'United States Minor Outlying Islands',
	'US': 'United States of America',
	'UY': 'Uruguay',
	'UZ': 'Uzbekistan',
	'VA': 'Vatican City State (Holy See)',
	'VC': 'St. Vincent & the Grenadines',
	'VE': 'Venezuela',
	'VG': 'British Virgin Islands',
	'VI': 'United States Virgin Islands',
	'VN': 'Viet Nam',
	'VU': 'Vanuatu',
	'WF': 'Wallis & Futuna Islands',
	'WS': 'Samoa',
	'YD': 'Democratic Yemen (no longer exists)',
	'YE': 'Yemen',
	'YT': 'Mayotte',
	'YU': 'Yugoslavia',
	'ZA': 'South Africa',
	'ZM': 'Zambia',
	'ZR': 'Zaire',
	'ZW': 'Zimbabwe',
	'ZZ': 'Unknown or unspecified country',
}

In [16]:
location=(30.253618,-81.554299)
reverseGeocode(location)
# name is city, admin1 is state, admin2 is county, cc is country

Loading formatted geocoded file...


[{'lat': '30.33218',
  'lon': '-81.65565',
  'name': 'Jacksonville',
  'admin1': 'Florida',
  'admin2': 'Duval County',
  'cc': 'US'}]

In [17]:
# # creates empty column for city names that gets its value from geocoding
tdor['city'] = ''
# for index,row in tdor.iterrows():
#     if row['latitude'] and row['longitude']:
#         geocodex = reverseGeocode((row['latitude'], row['longitude'])) 
#         tdor.loc[index,'city'] = geocodex[0]['name']
#         tdor.loc[index,'state_province'] = geocodex[0]['admin1']
#         tdor.loc[index,'location'] = geocodex[0]['admin2']
#         tdor.loc[index,'country'] = country_codes_to_country[geocodex[0]['cc']]



In [18]:
tdor.isna().sum()

name                 0
age               1366
photo             2349
photo_source      2507
date                 0
source_ref         481
location             0
state_province       0
country              0
latitude             0
longitude            0
category             1
cause_of_death       0
description          3
tweet             2215
permalink            0
year                 0
city                 0
dtype: int64

In [20]:
# dropping  columns that are not needed for our analysis
tdor=tdor.drop(columns = ['photo','photo_source','tweet','source_ref', 'permalink'])

In [21]:
# created city column so location column is unnecessary
tdor.drop(columns='location', inplace=True)

In [22]:
# exporting tdor to save our post-reverse geocode dataframe (as it took 5 hours..)
# tdor.to_csv('../Data/processed_data/tdor_presplit.csv',index=False)

# Splitting into Train and Test Sets

In [23]:
# splitting train data into a train,test 
train, test  = train_test_split(tdor, test_size=0.3, random_state=42)

# Imputing

In [24]:
# impute missing values for age by finding the median of age
# apply training set age median to testing set 
train['age'] = train['age'].fillna(train['age'].median())
test['age'] = test['age'].fillna(train['age'].median())

/var/folders/qn/34_2dqh118513z6jb88w8spw0000gn/T/ipykernel_39320/3404858917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['age'] = train['age'].fillna(train['age'].median())
/var/folders/qn/34_2dqh118513z6jb88w8spw0000gn/T/ipykernel_39320/3404858917.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['age'] = test['age'].fillna(train['age'].median())


In [25]:
# dropping all missing values
train.dropna(inplace=True)
test.dropna(inplace=True)

/Users/ranyasatkuna/opt/anaconda3/envs/dsi/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [26]:
# creating a binary column for violent deaths vs nonviolent for training set
train['violence']=train['category'].map({'violence': 1, 'uncategorised':0, 'suicide':0, 'medical':0,'custodial':0})

/var/folders/qn/34_2dqh118513z6jb88w8spw0000gn/T/ipykernel_39320/601105749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['violence']=train['category'].map({'violence': 1, 'uncategorised':0, 'suicide':0, 'medical':0,'custodial':0})


In [27]:
# creating a binary column for violent deaths vs nonviolent for testing set
test['violence']=test['category'].map({'violence': 1, 'uncategorised':0, 'suicide':0, 'medical':0,'custodial':0})

/var/folders/qn/34_2dqh118513z6jb88w8spw0000gn/T/ipykernel_39320/1319424010.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['violence']=test['category'].map({'violence': 1, 'uncategorised':0, 'suicide':0, 'medical':0,'custodial':0})


In [28]:
# checking for null values in training set
train.isna().sum()

name              0
age               0
date              0
state_province    0
country           0
latitude          0
longitude         0
category          0
cause_of_death    0
description       0
year              0
city              0
violence          0
dtype: int64

In [29]:
# checking for null values in testing set
test.isna().sum()

name              0
age               0
date              0
state_province    0
country           0
latitude          0
longitude         0
category          0
cause_of_death    0
description       0
year              0
city              0
violence          0
dtype: int64

In [30]:
# exporting training set 
# train.to_csv('../Data/processed_data/train.csv',index=False)

In [31]:
# exporting testing set
# test.to_csv('../Data/processed_data/test.csv',index=False)